In [1]:
from PIL import Image
import numpy as np

In [2]:
import os
import urllib
from urllib.request import urlretrieve
dataset = 'mnist.pkl.gz'
def reporthook(a,b,c):
    print("\rdownloading: %5.1f%%"%(a*b*100.0/c), end="")
    
if not os.path.isfile(dataset):
        origin = "https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz"
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset, reporthook=reporthook)

In [3]:
import gzip
import pickle
with gzip.open(dataset, 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

In [4]:
cog = len(np.unique(test_set[1]))

print("Length of train data: ",len(train_set[0]))
print("Length of test data: ",len(test_set[0]))
print("Class: ",cog)

Length of train data:  50000
Length of test data:  10000
Class:  10


In [5]:
# 設定好訓練及測試資料
train_X, train_y = train_set
test_X, test_y = test_set
# 設定成我們的格式
train_X = train_X[..., None]
test_X = test_X[..., None]



In [6]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
print(train_y[:20])
#print(train_X[:2])
showX(train_X[:20])

[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


In [7]:
def relu(x):
    return np.maximum(x, 0)
# 微分
def Drelu(x):
    return (x>0).astype('float32')

In [29]:
# 參考範例 softmax regression
W = np.random.normal(size=(25, 784))
W2 = np.random.normal(size=(10, 25))
b = np.random.normal(size=(25, 1))

c = np.random.normal(size=(10, 1))

n_data = train_X.shape[0]
# 紀錄 loss
loss_history = []
accuracy_history = []
for epoch in range(6000):    
    idx = np.random.choice(n_data, 500, replace=False)
    X = train_X[idx]
    y = train_y[idx]
    one_y = np.eye(10)[y][..., None]
    
    A = W @ X + b
    ta = relu(A)
    
    
    d = np.exp(W2 @ ta + c)
    q = d/d.sum(axis=(1,2), keepdims=True)
    loss = -np.log(q[range(len(y)), y]).mean()
    loss_history.append(loss)
    accuracy = (q.argmax(axis=1).ravel() == y).mean()
    accuracy_history.append(accuracy)
    if epoch%100 == 0:
        print(epoch, accuracy, loss)
    grad_W2_all = q - one_y
    
    grad_W2 = grad_W2_all @ ta.swapaxes(1,2)
    grad_W2 = grad_W2.mean(axis=0)
    #print(W2.T.shape)
    
    grad_ta_all = np.array([W2.T @ t for t in grad_W2_all])
    #grad_ta_all = W2.swapaxes(1,2) @ grad_w2_all

    grad_ta = grad_ta_all*Drelu(A)
    #print((1-pow(np.tanh(grad_ta_all),2)).shape)
    grad_b = grad_ta.mean(axis=0)
    
    grad_W = grad_ta @ X.swapaxes(1,2)
    grad_W = grad_W.mean(axis=0)
    #print(grad_W.shape)
    #grad_W = grad_W_all.mean(axis=0)
    #W -= grad_W
    #b -= grad_b    
    W -=  0.1*((5001-epoch)/5000)*grad_W
    W2 -=  0.1*((5001-epoch)/5000)*grad_W2
    b -= 0.1*((5001-epoch)/5000)*grad_b    
    c -= 0.1*((5001-epoch)/5000)*grad_W2_all.mean(axis=0)



0 0.064 61.08446741354032
100 0.476 2.2272008757339004
200 0.602 1.331182700931598
300 0.63 1.1836834581382
400 0.618 1.0717025278112269
500 0.654 1.0074608720234601
600 0.7 0.884798418723881
700 0.704 0.9062967530515122
800 0.728 0.7574738521606026
900 0.742 0.7490766613064129
1000 0.754 0.7077710803970363
1100 0.756 0.7908529108710495
1200 0.748 0.6967790638480146
1300 0.756 0.7359170110339925
1400 0.748 0.7236315589066933
1500 0.792 0.6309483670939722
1600 0.782 0.6365598976098019
1700 0.79 0.5757934499798719
1800 0.798 0.6078774994520525
1900 0.802 0.6443967559958371
2000 0.796 0.6221896744197627
2100 0.776 0.6773550517800325
2200 0.812 0.5776745706188229
2300 0.82 0.5990517081142185
2400 0.822 0.5797655200899063
2500 0.83 0.5437380454561941
2600 0.834 0.5484959587108198
2700 0.81 0.5975882561572524
2800 0.822 0.5489352328500132
2900 0.81 0.5998816975881532
3000 0.826 0.5545535330636281
3100 0.85 0.47788069142860995
3200 0.816 0.6282418727262823
3300 0.84 0.46208356228302744
3400 0

/Users/tonyguo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/Users/tonyguo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide
/Users/tonyguo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in maximum
  
/Users/tonyguo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


5300 0.106 nan
5400 0.128 nan
5500 0.116 nan
5600 0.094 nan
5700 0.094 nan
5800 0.122 nan
5900 0.102 nan


# mean-square-error

In [ ]:
# 參考範例 softmax regression
W = np.random.normal(size=(25, 784))
W2 = np.random.normal(size=(10, 25))
b = np.random.normal(size=(25, 1))

c = np.random.normal(size=(10, 1))

n_data = train_X.shape[0]
# 紀錄 loss
loss_history = []
accuracy_history = []
for epoch in range(6000):    
    idx = np.random.choice(n_data, 500, replace=False)
    X = train_X[idx]
    y = train_y[idx]
    one_y = np.eye(10)[y][..., None]
    
    A = W @ X + b
    ta = relu(A)
    
    d = W2 @ ta + c
    #d = np.exp(W2 @ ta + c)
    q = d/d.sum(axis=(1,2), keepdims=True)
    #loss = -np.log(q[range(len(y)), y]).mean()
    loss = np.array([(q[t]- one_y[t]).T@(q[t]- one_y[t]) for t in range(len(q))])
    #print(loss.mean(axis=1).shape)
    loss = ((loss.mean(axis=1))/len(one_y[0])).mean()
    #print(loss.mean().shape)
    loss_history.append(loss)
    accuracy = (q.argmax(axis=1).ravel() == y).mean()
    accuracy_history.append(accuracy)
    if epoch%100 == 0:
        print(epoch, accuracy, loss)
        #print("")
    #grad_W2_all = q - one_y
    
    #print((d*one_y).shape)
    grad_W2_all = (2/len(one_y[0]))*(d*one_y)
    #print(grad_W2_all.shape)
    grad_W2_all = grad_W2_all*np.array([(-1)*(q[t].T@one_y[t])*(q[t]-one_y[t]) for t in range(len(q))])
    
    grad_W2 = grad_W2_all @ ta.swapaxes(1,2)
    grad_W2 = grad_W2.mean(axis=0)
    #print(W2.T.shape)
    
    grad_ta_all = np.array([W2.T @ t for t in grad_W2_all])
    #grad_ta_all = W2.swapaxes(1,2) @ grad_w2_all

    grad_ta = grad_ta_all*Drelu(A)
    #print((1-pow(np.tanh(grad_ta_all),2)).shape)
    grad_b = grad_ta.mean(axis=0)
    
    grad_W = grad_ta @ X.swapaxes(1,2)
    grad_W = grad_W.mean(axis=0)
    #print(grad_W.shape)
    #grad_W = grad_W_all.mean(axis=0)
    #W -= grad_W
    #b -= grad_b    
    W -=  0.1*((5001-epoch)/5000)*grad_W
    W2 -=  0.1*((5001-epoch)/5000)*grad_W2
    b -= 0.1*((5001-epoch)/5000)*grad_b    
    c -= 0.1*((5001-epoch)/5000)*grad_W2_all.mean(axis=0)




0 0.082 190323.22517583327
100 0.1 0.17163747832973397
